# Procesamiento de Imágenes

El siguiente notebook tiene como objetivo realizar el procesamiento de imágenes correspondiente a paneles flexibles (banderas invertidas), las cuales han sido capturadas en vídeos bajo condiciones de luminosidad deficiente.

El dataset consta de varios frames correspondiente al vídeo anteriormente mencionado, los cuales son facilitados para su uso.

Para realizar el procesamiento de este tipo de imágenes, se realizaron 2 diferentes conjuntos de operaciones para llegar a un solo objetivo final, recolectar la mayor información posible de la silueta del filamento.

## Ambiente de Trabajo
Se importan las librerías a utilizar a lo largo del procesamiento de imágenes

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from math import sqrt,exp
from scipy import ndimage as ndi
from skimage.util import img_as_ubyte
from skimage.util import img_as_float

## Primer conjunto
Antes de empezar con el procesamiento se generó una función para recorrer ficheros, la cual recibe como parámetro el directorio al que se va a iterar para guardar la información necesaria y retorna como valor una lista de imágenes

In [2]:
#Directorio a recorrer
path_filamentos = '.\Dataset 1 - Filamentos\\'

def Ficheros(path):
    contenido = os.listdir(path)

    #Acumulador del nombre de las imagenes a utilizar
    imagenes = []

    #Se recorre el path para obtener los nombre y se los aggrega al acumulador, en este caso la lista
    for fichero in contenido:
        if os.path.isfile(os.path.join(path, fichero)) and fichero.endswith('.jpg'):
            imagenes.append(fichero)
            
    return imagenes        

### Eliminación de la base fija
Antes de este paso, se recomienda utilizar la GUI que se encuentra en el archivo 'TrackerPuntos.py' para delimitar la zona a eliminar. Una vez deilimitado los puntos se procedió a genera la función que eliminará la base fija.

In [3]:
#La variable fuera de la función es el directorio donde se guardaran las imagenes procesadas
path_sin_base = '.\Dataset 2 - Sin Base\\' 

#Se crea una función que nos permita eliminar el soporte del filamento
def eliminar_soporte(file_name):
    #Se abre la imagen
    new_img = cv2.imread(path_filamentos + file_name)

    #Se crean 2 cuadrados de diferente dimensión para poder eliminar la parte deseada
    new_img = cv2.rectangle(new_img, (750, 1327), (1745, 1725), (20, 25, 30), -1)
    new_img = cv2.rectangle(new_img, (500, 1590), (2000, 1725), (20, 25, 30), -1)

    #Se guardan los resultados
    pathw = path_sin_base + file_name
    cv2.imwrite(pathw, new_img)

#Se utiliza la función para obtener los fichero que se van a procesar
filamentos = Ficheros(path_filamentos)

#Se aplica la función a los ficheros
for im in filamentos:
    eliminar_soporte(im)

### Eliminación del ruido en el background de la imagen - Sharpening
Con la base fija retirada, se procedió a realizar la eliminación del ruido que se encuentra en el background de cada imagen, además de realizar la operación de 'Sharpening' y guardar los resultados

In [4]:
#Directorio a guardar el resultado
path_sin_ruido = '.\Dataset 3 - Sin Ruido\\' 

#Función para realizar el denoising
def denoising(file_name):
    
    #Se abre el archivo
    img = cv2.imread(path_sin_base + file_name)

    """Se utiliza la función fastNlMeansDenoisingColored, para poder eliminar en cierta
    cantidad el ruido que está dentro de la imagen que está pasando por la función"""
    dst = cv2.fastNlMeansDenoisingColored(img, 25, 21, 21, 21, 23)

    """Se crea un kernel el cual será utilizado para realizar un sharpening de la
    imagen luego de realizar el denoising"""
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    im = cv2.filter2D(dst, -1, kernel)

    #Se guardan los resultados
    pathw = path_sin_ruido + file_name
    cv2.imwrite(pathw, im)
    
#Se aplica la función a los ficheros
for im in filamentos:
    denoising(im)

### Filtro de la mediana - Ecualización Adaptativa
Antes de realizar esta operación se filtró la imagen por medio del uso de un filtro de la media, para continuar con la operación de ecualización adaptativa a escalas grises.

In [5]:
#Directorio a guardar el resultado
path_ecual = '.\Dataset 4 - Ecualizacion\\' 

#Se genera la función que va a realizar la ecualización
def ecualizer(file_name):
    
    #Se abre el archivo
    img = cv2.imread(path_sin_ruido + file_name, 0)

    #Se filtra la imagen con un filtro de la mediana con un kernel 11x11
    img = cv2.medianBlur(img, 11, 0)

    #Se realiza una ecualización de la imagen para poder captar mejor los constrastes
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl1 = clahe.apply(img)

    #Se guardan los resultados
    pathw = path_ecual + file_name
    cv2.imwrite(pathw, cl1)

#Se aplica la función a los ficheros
for img in filamentos:
    ecualizer(img)

### Umbralización en el espacio HSV

Nuevamente se requirió realizar un ‘trackbar’ para poder identificar los valores del umbral alto y bajo requeridos, ya sea para los valores de HSV (Hue, Saturation y Value), así poder identificar simplemente la parte correspondiente al filamento. (Vease la implementación del ‘trackbar’ en el archivo 'TrackerUmbral.py')

Con los valores de los umbrales ya definidos, se procedió a la implementación de la función de la umbralización.

In [6]:
#Directorio a guardar los resultados
path_hsv1 = '.\Dataset 5 - HSV 1\\' 

#Se genera la función que va a realizar la umbralización
def hsv1(file_name):
   
    #Se abre el archivo
    img = cv2.imread(path_ecual + file_name)

    #Se convierte la imagen de BGR a RGB pues, openCV maneja BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Se realiza un filtro de la mediana con un kernel de 11x11
    img = cv2.medianBlur(img, 11, 0)

    #Se convierte la imagen de BGR a HSV para realizar la umbralización
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    #Se definen los umbrales altos y bajos para realizar la umbralización
    umbral_bajo = (0, 0, 47)
    umbral_alto = (179, 255, 255)
    
    #Se realiza la umbralización
    mask = cv2.inRange(img_hsv, umbral_bajo, umbral_alto)
    
    #Se guardan los resultados
    pathw = path_hsv1 + file_name
    cv2.imwrite(pathw, mask)

    
#Se aplica la función a los ficheros
for img in filamentos:
    hsv1(img)

## Segundo Conjunto

En el segundo paso se retomó el dataset en el paso 4, además que este segundo conjunto consta con el uso de otro lenguaje de programación (MATLab) para ciertas operaciones particulares.

### Low Pass Filter
Se utilizó la transformada de Fourier, para pasar al dominio de la frecuencia. En este caso al tratarse de pixeles, se utilizó una transformada discreta al no tratarse de valores continuos. El algoritmo a implementar fue el ‘Fast Fourier Transform’ el cual se encuentra disponible en la librería ‘scipy’ de Python. Esta operación permitió que se realizara un ‘low pass filter’ el cual permite que los bordes tengan una mejor definición y así poder segmentarlos de mejor manera. 

In [7]:
#Directorio a guardar los resultados
path_LPF = '.\Dataset 6 - FFT\\' 

#Se crea la función que para realizar el filtro
def lpf(file_name):
    
    #Distancia
    def distance(point1,point2):
        return sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)
    
    #Filtro ideal
    def idealFilterLP(D0,imgShape):
        base = np.zeros(imgShape[:2])
        rows, cols = imgShape[:2]
        center = (rows/2,cols/2)
        for x in range(cols):
            for y in range(rows):
                if distance((y,x),center) < D0:
                    base[y,x] = 1
        return base
    
    #Se abre el fichero
    img = cv2.imread(path_ecual + file_name, 0)
    
    #Fast Fourier Transform
    original = np.fft.fft2(img)
    center = np.fft.fftshift(original)
    
    #Low pass filter
    LowPass = idealFilterLP(50,img.shape)
    LowPassCenter = center * idealFilterLP(50,img.shape)
    LowPass = np.fft.ifftshift(LowPassCenter)
    
    #Operación inversa
    inverse_LowPass = np.fft.ifft2(LowPass)
    
    #Se pasa del dominio de la frecuencia al discreto
    filtered_img = np.abs(inverse_LowPass)
    filtered_img -= filtered_img.min()
    filtered_img = filtered_img*255 / filtered_img.max()
    filtered_img = filtered_img.astype(np.uint8)
    
    #Se guardan los resultados
    pathw = path_LPF  + file_name
    cv2.imwrite(pathw, filtered_img)

#Se aplica la función a los ficheros
for img in filamentos:
    lpf(img)

### Umbralización en el espacio HSV
Se realizó lo mismo que en la primera umbralización del Conjunto 1 de operaciones, con la variación en los umbrales, pues el análisis ahora es a los frames que pasaron por un 'Low pass Filter'.

In [8]:
#Directorio a guardar los resultados
path_hsv2 = '.\Dataset 7 - HSV 2\\' 

#Se define la función que va a realizar la umbralización
def hsv2(file_name):
    
    #Se abre el fichero
    img = cv2.imread(path_LPF + file_name)

    #Se convierte la imagen de BGR a RGB pues, openCV maneja BGR
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    #Se realiza un filtro de la mediana con un kernel de 11x11
    img = cv2.medianBlur(img, 11, 0)

    #Se convierte la imagen de BGR a HSV para realizar la umbralización
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    umbral_bajo = (0, 0, 150)
    umbral_alto = (179, 255, 255)
    
    #Se realiza la umbralización
    mask = cv2.inRange(img_hsv, umbral_bajo, umbral_alto)

    pathw = path_hsv2 + file_name
    cv2.imwrite(pathw, mask)

#Se aplica la función a los ficheros
for img in filamentos:
    hsv2(img)

### Superposición de resultados
Una vez obtenido el resultado del paso anterior, los dataset correspondientes a la umbralización de la operación 1 y operación 2 son superpuestos, pues información que no pudo ser segmentada dentro de la operación 1 lo fue posible en la operación 2 y viceversa.

In [9]:
#Directorio a guardar los resultados
path_merge = '.\Dataset 8 - Merge\\' 

#Directorios a abrir los archivos a combinar
path_hsv1 = '.\Dataset 5 - HSV 1\\'
path_hsv2 = '.\Dataset 7 - HSV 2\\'

#Se define la función que va a superponer los resultados
def merge(file_name):

    #Se abren instancias de las imagenes a combinar
    img = Image.open(path_hsv1 + file_name)
    background = Image.open(path_hsv2 + file_name)
    
    #Se realiza la superposición
    background.paste(img, (0, 0), img)
    
    #Se guardan resultados
    pathw = path_merge + file_name
    background.save(pathw)
    
#Se aplica la función a los ficheros
for img in filamentos:
    merge(img)

### Dilation
Se realizó la operación de dilatación, lo cual permitió unir ciertas partes de los filamentos segmentados que se encontraban dispersas.

In [10]:
#Directorio a guardar los resultados
path_dil = '.\Dataset 9 - Dilatacion\\' 

#Se define la función que va a realizar la operación de dilatación
def dilation(file_name):
    
    #Se abre la imagen
    img = cv2.imread(path_merge + file_name)
    
    #Se define el kernel con el que se va a trabajar
    kernal = np.ones((13, 13), np.uint8)
    
    #Se aplica la operación morfológica y sus respectivas iteraciones
    dilation = cv2.dilate(img, kernal, iterations=7)
    
    #Se guardan resultados
    pathw = path_dil + file_name
    cv2.imwrite(pathw, dilation)

#Se aplica la función a los ficheros
for img in filamentos:
    dilation(img)

### Erosion - Blob Analysis
El pre-procesamiento de la erosión se lo realizó para evitar que ciertos ‘blobs’, sigan en contacto con la figura del filamento y evitar que esto no permita segmentar de correcta la figura de las banderas.

Una vez finalizado el pre-procesamiento, se procedió a eliminar los ‘blobs’ que se encontraban alrededor de los frames analizados, pues este ruido evitaría que la operación posterior se lleve a cabo de una manera eficiente, pues es requerida simplemente la forma del filamento.

In [11]:
#Directorio a guardar los resultados
path_blob= '.\Dataset 10 - Blob Analysis\\' 

#Se define la función que va a realizar la operación mencionada
def blob(file_name):
    
    #Se abre el fichero
    img = cv2.imread(path_dil + file_name)
    
    #Se define el kernel con el que se va a realizar la erosion
    kernal = np.ones((3, 3), np.uint8)
    
    #Se aplica la operacion morfológica y sus respectivas iteraciones
    erosion = cv2.erode(img, kernal, iterations=3)
    
    """
    Se convierte la variable debido a que la librería 'scipy' no trabaja con variables tipo 'uint8' como 
    lo hace la libería 'OpenCV'
    """
    image = img_as_float(erosion)
    
    #Se definen los parámetros para realizar la segmentación de areas
    label_objects, nb_labels = ndi.label(image)
    sizes = np.bincount(label_objects.ravel())
    
    #Se define las áreas a eliminar
    mask_sizes = sizes > 350000
    mask_sizes[0] = 0
    clean = mask_sizes[label_objects]
    
    #El resultado se vuelve a transformar a valor tipo 'uint8'
    clean_2 = img_as_ubyte(clean)
    
    #Se guardan resultados
    pathw = path_blob + file_name
    cv2.imwrite(pathw, clean_2)

#Se aplica la función a los ficheros  
for img in filamentos:
    blob(img)

### Continuacion
A partir de este paso fue requerida la implementación del lenguaje de programación de MATLab, pues para las operaciones de esqueletonización y eliminación de 'spurs' se requirió el uso de la 'Image Processing Toolbox' de MATLab debido a las funciones que permitían un mejor desempeño para realizar las operaciones anteriormente mencionadas. Para ver su implementación revisar el archivo 'Esqueletonizacion.m' en el repositorio.